*train*-transliteration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=1fc1497fc6b9f18c39426b225f5062973bc76b8baba3a9184ddd53dd1913b5df
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-4-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-5-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd


In [ ]:
bodo_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/train_BO_AH_HASOC2023.csv')

In [ ]:
len(bodo_train)

1679

In [ ]:
bodo_train['task_1'].value_counts()

HOF    998
NOT    681
Name: task_1, dtype: int64

In [ ]:
bodo_train['clean_text'] = bodo_train['text'].apply(lambda x:emo(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:remove_urls(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train

,S. No.,text,task_1,clean_text
0,1,गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन,NOT,गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1,2,निखावरि सुबुंफोरा सिखाव,HOF,निखावरि सुबुंफोरा सिखाव
2,3,मा बिमा ख'र' परिबर्थननि खोथा फैखो बेयाव मोसौ,HOF,मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3,4,थोद जामबा सैमा साला मा मिसेस जाखो बेलाय,HOF,थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4,5,माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला,HOF,माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
...,...,...,...,...
1674,1675,नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा,HOF,नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा
1675,1676,एै मावजि लाब गैया दानो बनद खालामनायनि खोथा बुंबानो गाबोननो गरनैसो होलाय होनबानो जाबाय लागाव बुंग्रासो जागोन नोंसोरबायदि मावजि सादुफोरा,HOF,एै मावजि लाब गैया दानो बनद खालामनायनि खोथा बुंबानो गाबोननो गरनैसो होलाय होनबानो जाबाय लागाव बुंग्रासो जागोन नोंसोरबायदि मावजि सादुफोरा
1676,1677,सिखला फुरकव रपे खलामनांगव,HOF,सिखला फुरकव रपे खलामनांगव
1677,1678,सेनदेल खुबै नांगौलै सालाफोरखौ,HOF,सेनदेल खुबै नांगौलै सालाफोरखौ


In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
!git clone https://github.com/libindic/Transliteration.git

Cloning into 'Transliteration'...
remote: Enumerating objects: 354, done.
remote: Total 354 (delta 0), reused 0 (delta 0), pack-reused 354
Receiving objects: 100% (354/354), 908.74 KiB | 7.21 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [ ]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!python -m pip install --user virtualenv

  Obtaining dependency information for virtualenv from https://files.pythonhosted.org/packages/14/19/e266f07cf55155d5f45170bbe08c486d8a9a9ae17bc8983acb1c019a8dd4/virtualenv-20.24.2-py3-none-any.whl.metadata
  Obtaining dependency information for distlib<1,>=0.3.7 from https://files.pythonhosted.org/packages/43/a0/9ba967fdbd55293bacfc1507f58e316f740a3b231fc00e3d86dc39bc185a/distlib-0.3.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 17.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!python -m virtualenv env

created virtual environment CPython3.10.12.final.0-64 in 2079ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source env/bin/activate

In [ ]:
pwd

'/content'

In [ ]:
!pip install -r '/content/Transliteration/requirements.txt'

  Cloning https://github.com/libindic/normalizer.git to /tmp/pip-install-fxconvve/libindic-normalizer_4974d5f3480f401d8e73128671a6d8e8
  Running command git clone --filter=blob:none --quiet https://github.com/libindic/normalizer.git /tmp/pip-install-fxconvve/libindic-normalizer_4974d5f3480f401d8e73128671a6d8e8
  Resolved https://github.com/libindic/normalizer.git to commit 88447f611e44ddfc9e9c6696650671afc10c25ae
  Preparing metadata (setup.py) ... done
Ignoring repoze.lru: markers 'python_version < "3"' don't match your environment
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-utils: filename=libindic_utils-1.0.3-py3-none-any.whl size=9686 sha256=79cfef1c6d01b857e4409e090fcef29ab04a5a9cd36771b99aa5bf61f6c6081d
  Stored in directory: /root/.cache/pip/wheels/a9/50/5f/64d267b1b00db903376596b944d6d7ca80ad721254d6eff902
  Created wheel for libindic_normalizer: filename=libindic_normalizer-0.0.0-py2.py3-none-any.whl size=6359 sha256=aa843b9ff86775b5f193fb80a5f7c908d4

In [ ]:
!pip install  '/content/Transliteration/.'

Processing ./Transliteration
  Preparing metadata (setup.py) ... done
  Created wheel for libindic.transliteration: filename=libindic.transliteration-0.4.1-py3-none-any.whl size=851870 sha256=bd4a56f618fe82555b74cf3c121a58a00dcd20188b1092b74e429c56f8d31ac1
  Stored in directory: /tmp/pip-ephem-wheel-cache-vwmc4dpi/wheels/94/ea/8b/a1558e8a38a7980bb506204694fd26148d79d71167f025eb79
Successfully built libindic.transliteration


In [ ]:
from libindic.transliteration import getInstance


In [ ]:
ldata=bodo_train.clean_text
ldata=ldata.tolist()

In [ ]:
len(ldata)

1679

In [ ]:
fin=[]
for i in ldata:
  t = getInstance()
  t_text = t.transliterate(str(i), "en_IN")
  fin.append(t_text)
len(fin)

1679

In [ ]:
df=pd.DataFrame(fin)
df=df.rename(columns={0:'texttranslit'})
df.head(20)

,texttranslit
0,goodaav khaamaani maavao boolaa noo saanase derahaa thaaragoon
1,nikhaavari subumphooraa sikhaav
2,maa bimaa khar paribarthanani khoothaa phaikhoo beyaav moosou
3,thood jaamabaa saimaa saalaa maa mises jaakhoo belaay
4,maakhou bakibaay thaadoom noom booramaa phaanathaa dam damblaa khaamaani maavanaa dinthi bini unaav bakhi saimaa saalaa
5,deenavee aaeepeeel jubbgvam
6,souyoo aanoo yaar khaabhanyi
7,laaji roongai booramaa phaanathaa dipen bar saalaa gaavasoor haarini khoothaa bumnoo roongaa aaithim soolaagraa nilaaj phoor maalaayaa soomkhaarinaayaavaboo hemthaa moosouphoor
8,deenavee javamganee proojekt khav kalamathaaragun aaamg
9,maani jaahoonaav gilirasin besaadaphooraa gaahaai phaarase thaabasin daavabaainoo haayoo


In [ ]:
#To split train and test
X_train, X_test, y_train, y_test = train_test_split(df.texttranslit, bodo_train.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((1259,), (420,))

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='char', ngram_range=(1,3))
tfidf_vect.fit(X_train)
X_train = tfidf_vect.transform(X_train)
X_test = tfidf_vect.transform(X_test)

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(X_train, y_train)

y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))

Test Accuracy: 78.5714

               precision    recall  f1-score   support

         HOF       0.76      0.91      0.82       233
         NOT       0.84      0.64      0.73       187

    accuracy                           0.79       420
   macro avg       0.80      0.77      0.77       420
weighted avg       0.80      0.79      0.78       420



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(X_train, y_train)
y_pred = eclf.predict(X_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))